## Testes da rotina da classe das Equações de Mackey-Glass

## 1. Importando as bibliotecas necessárias

### Bibliotecas obrigatórias

In [1]:
import numpy as np
from jitcdde import jitcdde, y, t

%matplotlib widget
import matplotlib.pyplot as plt

import seaborn as sns # a biblioteca 'seaborn' contém vários estilos para os gráficos do 'matpĺotlib'

# agora, melhoramos a qualidade de saida e de visualizacao da imagem 
# alem de mudar a fonte padrao para uma do latex
sns.set_style("ticks")
plt.rcParams['savefig.dpi'] = 200
plt.rcParams["figure.dpi"] = 100

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})

plt.style.use('dark_background')

### Bibliotecas não-obrigatórias

In [2]:
import sys 
sys.path.insert(0, '../../scripts')
import mackeyglassequations as mgeq

## 2. Definindo a função (debug)

Não é necessário executar essa linha se você estiver importando o arquivo com a classe.

In [95]:
class MackeyGlass:
    
    def __init__(self, p_inicial, gamma=0.1, beta=0.2, theta=1, tau=30, n=30, dt=0.0001):
        """
        Descrição:
        ----------
        Construtor da classe 'SistemaLorenz'

        Parâmetros:
        -----------
        p_inicial: int ou float
            Valor inicial da concentração de glóbulos vermelhos no sangue
        gamma: int ou float
            Parâmetro das Equações de Mackey-Glass
        beta: int ou float
            Parâmetro das Equações de Mackey-Glass
        theta: int ou float
            Parâmetro das Equações de Mackey-Glass
        tau: int
            Parâmetro das Equações de Mackey-Glass
        n: int
            Parâmetro das Equações de Mackey-Glass
        dt: float
            Tamanho do diferencial de tempo que iremos utilizar nos cálculos, ou seja, a resolução temporal de nossa solução
            
        Retorna:
        --------
        Nada
        """
        
        if not ((type(p_inicial) is int) | (type(p_inicial) is float) & (p_inicial > 0)):
            raise TypeError("0 valor da concentração de glóbulos vermelhos no sangue deve ser um int ou float positivo!")
        
        if not (((type(gamma) is int) | (type(gamma) is float)) & (gamma > 0)):
            raise TypeError("Gamma deve ser um int ou float positivo!")

        if not (((type(beta) is int) | (type(beta) is float)) & (beta > 0)):
            raise TypeError("Beta deve ser um int ou float positivo!")

        if not (((type(theta) is int) | (type(theta) is float)) & (theta > 0)):
            raise TypeError("Theta deve ser um int ou float positivo!")
                
        if not (((type(tau) is int) | (type(tau) is float)) & (tau > 0)):
            raise TypeError("Tau deve ser um int ou float positivo!")
                
        if not ((type(n) is int) & (n > 0)):
            raise TypeError("n deve ser um int positivo!")
            
        if not ((type(dt) is float) & (dt > 0)):
            raise TypeError("dt deve ser um float positivo!")
        
        self._p_inicial = p_inicial
        self._gamma = gamma
        self._beta = beta
        self._theta = theta
        self._tau = tau
        self._n = n
        self._dt = dt
        
        p_delays = np.random.rand(tau, 1)
        self._p_delays = p_delays
        pass
    
    def _equacoes(self):
        """
        Descrição:
        ----------
        Função interna que retorna as equações de Mackey-Glass calculadas no instante t atual
        
        Parâmetros:
        -----------
        estado_atual: np.ndarray
            Vetor das posições (p_atual, p_delay) atuais do sistema    
        t: float
            Instante t no qual estamos calculando as derivadas
            
        Retorna:
        --------
        As equações de Mackey-Glass para os parâmetros estimados na forma de um array
        """     
        
        gamma = self._gamma
        beta = self._beta
        theta = self._theta
        tau = self._tau
        n = self._n
        
        dp_dt = (beta*((theta)**n))/(((theta)**n) + (y(0,t-tau)**n)) - gamma*y(0)
        dq_dt = (beta*((theta)**n)*y(0,t-tau))/(((theta)**n) + (y(0,t-tau)**n)) - gamma*y(0)
        return [dq_dt]
    
    def calcular(self, t_inicial, t_final):
        """
        Descrição:
        ----------
        Evolui as equações de Mackey-Glass com base nas condições iniciais configuradas, para um t indo de t_inicial até t_final, com n_instantes calculados
        
        Parâmetros:
        -----------
        t_inicial: int
            Dia em que iniciamos os cálculos
        t_final: int
            Dia em que terminamos os cálculos
            
        Retorna:
        --------
        Um vetor com as soluções estimadas e um vetor com os instantes temporais utilizados
        """
        
        if not ((type(t_inicial) is int) & (t_inicial >= 0)):
            raise TypeError("t_inicial deve ser um int não nulo!")
            
        if not ((type(t_final) is int) & (t_final > 0)):
            raise TypeError("t_final deve ser um int positivo!")
        
        gamma = self._gamma
        beta = self._beta
        theta = self._theta
        tau = self._tau
        n = self._n
        p_anteriores = self._p_delays
        dt = self._dt
        
        p_iniciais = np.append(p_anteriores, self._p_inicial)
        p_anteriores_derivada = np.zeros(len(p_iniciais))
        t_anteriores = np.arange(-len(p_iniciais)+1, 1, 1)
        condicoes_iniciais = []
        
        for i in range(0, len(p_iniciais), 1):
            condicao = (t_anteriores[i], p_iniciais[i], p_anteriores_derivada[i])
            condicoes_iniciais.append(condicao)
        
        equacoes = self._equacoes()
        DDE = jitcdde(equacoes)
        
        DDE.add_past_points(condicoes_iniciais)
        DDE.step_on_discontinuities()
        
        n_instantes = int((t_final - t_inicial)/dt)        
        instantes_temporais = DDE.t + np.linspace(t_inicial, t_final, n_instantes)
        
        solucoes = []
        for t in instantes_temporais:
            solucoes.append(DDE.integrate(t))
        
        return solucoes

## 3. Testando a classe

In [96]:
t_inicial = 0
t_final = 800
dt = 0.001

In [97]:
p_inicial = 0.8

In [98]:
macglass = MackeyGlass(p_inicial, dt=dt)

In [99]:
solucoes = macglass.calcular(t_inicial = t_inicial, t_final = t_final)

Generating, compiling, and loading C code.


ModuleNotFoundError: No module named 'sympy'

In [100]:
solucoes

[(-30, 0.46159780723314503, 0.0),
 (-29, 0.8301661749645505, 0.0),
 (-28, 0.317598879807264, 0.0),
 (-27, 0.668834178411854, 0.0),
 (-26, 0.7716748957652151, 0.0),
 (-25, 0.3185679226744692, 0.0),
 (-24, 0.8818574549640174, 0.0),
 (-23, 0.4740850048135493, 0.0),
 (-22, 0.12597506839890205, 0.0),
 (-21, 0.587932890168456, 0.0),
 (-20, 0.9635631569224554, 0.0),
 (-19, 0.15682969424271287, 0.0),
 (-18, 0.1350903799012414, 0.0),
 (-17, 0.5349036059177915, 0.0),
 (-16, 0.9977601671382332, 0.0),
 (-15, 0.14990491482071477, 0.0),
 (-14, 0.9638283343246318, 0.0),
 (-13, 0.8364040488954624, 0.0),
 (-12, 0.815457248016634, 0.0),
 (-11, 0.4859022842194256, 0.0),
 (-10, 0.9056932817150678, 0.0),
 (-9, 0.5507998833994451, 0.0),
 (-8, 0.958149795036398, 0.0),
 (-7, 0.8437531377047851, 0.0),
 (-6, 0.8787206782882337, 0.0),
 (-5, 0.5909639419494408, 0.0),
 (-4, 0.679556198987999, 0.0),
 (-3, 0.988034887247574, 0.0),
 (-2, 0.5926772039727977, 0.0),
 (-1, 0.7736508339957815, 0.0),
 (0, 0.8, 0.0)]

In [36]:
p = solucoes[:, 0]
q = solucoes[:, 1]

NameError: name 'solucoes' is not defined

### 3.a) Série Temporal de Mackey-Glass

In [ ]:
fig, ax = plt.subplots(2)
fig.suptitle("Séries temporais de 0 a 100 segundos das coordenadas xyz do Sistema de Lorenz")
ax[0].plot(instantes_temporais, p, color='DeepSkyBlue')

ax[0].set_ylabel('x(t)')
ax[0].set_xlabel('t')
ax[0].set_xlim(0,)
    
ax[0].grid(True)

ax[1].plot(instantes_temporais, q, color='LightCoral')
ax[1].set_ylabel('y(t)')
ax[1].set_xlabel('t')
ax[1].set_xlim(0,)
ax[1].grid(True)

fig.tight_layout()
sns.despine()
plt.show()

### 3.b) Atrator de Mackey-Glass